In [1]:
import sys
!{sys.executable} -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!{sys.executable} -m pip install silero-vad pandas pydub openpyxl







Looking in indexes: https://download.pytorch.org/whl/cpu



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
from pydub import AudioSegment
from pathlib import Path
import pandas as pd
from silero_vad import VADIterator
import os
import librosa

c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [4]:
def merge_close_segments(segments, max_silence_sec=5, sr=16000):
    merged = []
    for seg in segments:
        if not merged:
            merged.append(seg)
        else:
            prev = merged[-1]
            # se la pausa tra prev ed attuale è minore di max_silence_sec, uniscili
            if (seg['start'] - prev['end']) / sr <= max_silence_sec:
                prev['end'] = seg['end']
            else:
                merged.append(seg)
    return merged


In [112]:
# CONFIGURAZIONE
# ================================================
input_dir = Path("C:/Users/gioel/Desktop/patient_GAN_vad/control_CM08/pre_vad/")
output_dir = Path("C:/Users/gioel/Desktop/patient_GAN_vad/control_CM08/")
output_dir.mkdir(exist_ok=True)
sr = 16000
results = []
failed_files = []

In [51]:
device = 'cpu'  # CPU-only
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad', force_reload=False)
(get_speech_timestamps, _, read_audio, _, _) = utils
vad_iterator = VADIterator(model, sampling_rate=sr)


Using cache found in C:\Users\gioel/.cache\torch\hub\snakers4_silero-vad_master


In [52]:

def sec_to_minsec(sec):
    m = int(sec // 60)
    s = int(sec % 60)
    return f"{m}.{s:02d}"

In [113]:
for subfolder in input_dir.iterdir():
    if subfolder.is_dir():
        for wav_file in subfolder.glob("*.wav"):
            waveform = read_audio(str(wav_file), sampling_rate=sr) 
             
            # Ottieni segmenti VAD 
            speech_timestamps = get_speech_timestamps(waveform, model, sampling_rate=sr, threshold=0.3)

            if not speech_timestamps:
                failed_files.append({'file': wav_file.name, 'path': str(wav_file)})
                continue
            
             # Unisci segmenti separati da silenzi brevi
            speech_timestamps = merge_close_segments(speech_timestamps, max_silence_sec=5, sr=sr)
            
            # Calcola probabilità per chunk per ciascun segmento
            for seg in speech_timestamps:
                start_sample = seg['start']
                end_sample = seg['end']
                segment_waveform = waveform[start_sample:end_sample]

                # Calcola probabilità a chunk
                speech_probs = []
                window_size_samples = 512 if sr == 16000 else 256
                for i in range(0, len(segment_waveform), window_size_samples):
                    chunk = segment_waveform[i:i+window_size_samples]
                    if len(chunk) < window_size_samples:
                        break
                    prob = model(chunk, sr).item()
                    speech_probs.append(prob)
                vad_iterator.reset_states()  # reset tra segmenti

                # Aggiungi score medio al segmento
                seg['speech_prob'] = speech_probs
                seg['score'] = sum(speech_probs)/len(speech_probs) if speech_probs else 0

            audio = AudioSegment.from_file(wav_file)
            
            # Salva ogni segmento e aggiungi info per Excel
            for i, seg in enumerate(speech_timestamps, start=1):
                start_ms = int(seg['start'] / sr * 1000)
                end_ms = int(seg['end'] / sr * 1000)
                score = sum(seg['speech_prob']) / len(seg['speech_prob'])
                clip = audio[start_ms:end_ms]

                # Salva segmento
                out_subdir = output_dir / subfolder.name
                out_subdir.mkdir(exist_ok=True)
                out_path = out_subdir / f"{wav_file.stem}_segment{i}.wav"
                clip.export(out_path, format="wav")

                # Calcola durata in secondi e minuti.secondi
                duration_sec = (end_ms - start_ms) / 1000
                duration_minsec = sec_to_minsec(duration_sec)

                # Aggiungi al dataframe
                results.append({
                    'file': wav_file.name,
                    'segment': i,
                    'start_sec': start_ms/1000,
                    'end_sec': end_ms/1000,
                    'duration_sec': duration_sec,
                    'duration_min.sec': duration_minsec,
                    'score': seg['score'],
                })


c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\silero_vad\utils_vad.py:139: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  list_backends = torchaudio.list_audio_backends()
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\silero_vad\utils_vad.py:150: UserWarning: torchaudio.sox_effects.sox_effects.apply_effects_file has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  wav, sr = torc

In [114]:
# SALVA EXCEL
# ================================================
df = pd.DataFrame(results)
df.to_excel(output_dir / "vad_segments.xlsx", index=False)

df_failed = pd.DataFrame(failed_files)
df_failed.to_excel(output_dir / "VAD_failed_files.xlsx", index=False)
print(f"✅ VAD completato. Excel salvato in: {output_dir / 'vad_segments.xlsx'}")

✅ VAD completato. Excel salvato in: C:\Users\gioel\Desktop\patient_GAN_vad\control_CM08\vad_segments.xlsx


In [2]:
# Percorso della cartella contenente i file
cartella = "C:/Users/gioel/Desktop/patient_GAN_vad/patient_M08/M08"  # Sostituisci con il tuo percorso

# Ciclo su tutti i file nella cartella
for filename in os.listdir(cartella):
    if filename.endswith(".wav") and "_segment1" in filename:
        nuovo_nome = filename.replace("_segment1", "")
        vecchio_percorso = os.path.join(cartella, filename)
        nuovo_percorso = os.path.join(cartella, nuovo_nome)
        os.rename(vecchio_percorso, nuovo_percorso)
        print(f"Rinominato: {filename} -> {nuovo_nome}")

Rinominato: B1_C10_segment1.wav -> B1_C10.wav
Rinominato: B1_C11_segment1.wav -> B1_C11.wav
Rinominato: B1_C12_segment1.wav -> B1_C12.wav
Rinominato: B1_C13_segment1.wav -> B1_C13.wav
Rinominato: B1_C14_segment1.wav -> B1_C14.wav
Rinominato: B1_C15_segment1.wav -> B1_C15.wav
Rinominato: B1_C16_segment1.wav -> B1_C16.wav
Rinominato: B1_C17_segment1.wav -> B1_C17.wav
Rinominato: B1_C18_segment1.wav -> B1_C18.wav
Rinominato: B1_C19_segment1.wav -> B1_C19.wav
Rinominato: B1_C1_segment1.wav -> B1_C1.wav
Rinominato: B1_C2_segment1.wav -> B1_C2.wav
Rinominato: B1_C3_segment1.wav -> B1_C3.wav
Rinominato: B1_C4_segment1.wav -> B1_C4.wav
Rinominato: B1_C5_segment1.wav -> B1_C5.wav
Rinominato: B1_C6_segment1.wav -> B1_C6.wav
Rinominato: B1_C7_segment1.wav -> B1_C7.wav
Rinominato: B1_C8_segment1.wav -> B1_C8.wav
Rinominato: B1_C9_segment1.wav -> B1_C9.wav
Rinominato: B1_CW100_segment1.wav -> B1_CW100.wav
Rinominato: B1_CW10_segment1.wav -> B1_CW10.wav
Rinominato: B1_CW11_segment1.wav -> B1_CW11.wa

In [49]:
import os
import sys
from pathlib import Path

def confronta_cartelle(cartella_a, cartella_b):
    """
    Trova i file presenti in cartella_a che non sono in cartella_b.
    
    Args:
        cartella_a (str | Path): Il percorso della prima cartella.
        cartella_b (str | Path): Il percorso della seconda cartella.
        
    Returns:
        list: Una lista dei nomi dei file (senza percorso) unici in cartella_a.
    """
    
    # Converto in stringhe nel caso siano oggetti Path
    cartella_a = str(cartella_a) 
    cartella_b = str(cartella_b)
    
    # --- 1. Verifica Esistenza Cartelle ---
    if not os.path.isdir(cartella_a):
        print(f"ERRORE: La cartella A '{cartella_a}' non esiste o non è una cartella.")
        return []
    if not os.path.isdir(cartella_b):
        print(f"ERRORE: La cartella B '{cartella_b}' non esiste o non è una cartella.")
        return []

    print(f"Analisi in corso tra A: '{cartella_a}' e B: '{cartella_b}'...")
    print("-" * 40)
    
    # --- 2. Ottieni la Lista dei File ---
    # La list comprehension filtra solo i file (ignorando le sottocartelle)
    # e la conversione a set velocizza il confronto.
    try:
        files_a = {f for f in os.listdir(cartella_a) if os.path.isfile(os.path.join(cartella_a, f))}
        files_b = {f for f in os.listdir(cartella_b) if os.path.isfile(os.path.join(cartella_b, f))}

    except PermissionError as e:
        print(f"ERRORE di Permesso: Non è possibile accedere a una delle cartelle. Dettagli: {e}")
        return []
        
    # --- 3. Confronta i Set ---
    # Set difference: trova gli elementi in files_a che NON sono in files_b
    files_unici_in_a = files_a - files_b
    
    return sorted(list(files_unici_in_a))


# --- PARTE PRINCIPALE: Esempio di Utilizzo per JUPYTER NOTEBOOK ---

# **MODIFICA QUESTI PERCORSI CON LE TUE CARTELLE**
CARTELLA_UNO = Path("C:/Users/gioel/Desktop/patient_GAN_vad/patient_M05/pre_vad/M05") 
CARTELLA_DUE = Path("C:/Users/gioel/Desktop/patient_GAN_vad/patient_M05/M05") 

# Logica per sys.argv disabilitata per prevenire SystemExit nei Notebook
# Se esegui da terminale, puoi ripristinare il blocco sys.argv per accettare argomenti:
# if len(sys.argv) == 3:
#     CARTELLA_UNO = sys.argv[1]
#     CARTELLA_DUE = sys.argv[2]
# elif len(sys.argv) != 1:
#     print("Utilizzo: python nome_script.py <cartella_1> <cartella_2>")
#     sys.exit(1)

risultato = confronta_cartelle(CARTELLA_UNO, CARTELLA_DUE)

print("\n## 🔎 RISULTATO DEL CONFRONTO ##")
print("-" * 30)

if risultato:
    print(f"🚨 Trovati **{len(risultato)}** file in '{CARTELLA_UNO}' che NON sono presenti in '{CARTELLA_DUE}':")
    for file in risultato:
        print(f"- {file}")
else:
    print(f"✅ Nessun file aggiuntivo trovato. Le cartelle contengono gli stessi file (secondo il confronto A - B).")

Analisi in corso tra A: 'C:\Users\gioel\Desktop\patient_GAN_vad\patient_M05\pre_vad\M05' e B: 'C:\Users\gioel\Desktop\patient_GAN_vad\patient_M05\M05'...
----------------------------------------

## 🔎 RISULTATO DEL CONFRONTO ##
------------------------------
🚨 Trovati **5** file in 'C:\Users\gioel\Desktop\patient_GAN_vad\patient_M05\pre_vad\M05' che NON sono presenti in 'C:\Users\gioel\Desktop\patient_GAN_vad\patient_M05\M05':
- B3_CW27.wav
- B3_CW69.wav
- B3_CW72.wav
- B3_CW78.wav
- B3_CW79.wav
